In [1]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data/data.xlsx', sheet_name='reduced totals')
# print(df)

In [4]:
## LOADING IN DATASETS

dataset = Path.cwd().joinpath("SongEmotionDataset")
datasheet = Path.cwd().joinpath("data") # for csua

#emotion labels
label_loc = datasheet.joinpath("data.xlsx")
wb = xlrd.open_workbook(label_loc) 
sheet = wb.sheet_by_index(2)

#emotion arr
emotions = ["amazement", "calmness", "power", "joyful activation", "sadness"]

In [26]:
train_percentage = 0.8
allowed_exceedance = 0

train_song = []
test_song = []
train_emotion = []
test_emotion = []

row_indexes = np.arange(1,401)
np.random.shuffle(row_indexes)

# train_indexes = [row_indexes[i] for i in range(len(row_indexes)) if i < len(row_indexes)*train_percentage]
# test_indexes = [row_indexes[i] for i in range(len(row_indexes)) if i >= len(row_indexes)*train_percentage]

def get_data(indexes):
    song = []
    emotion = []
    
    totals = torch.zeros(len(emotions), device=device).float()
    for x in indexes:    
        row = torch.tensor([sheet.cell_value(x, 2 + j) for j in range(5)], device=device).float()
        totals += F.softmax(row)

    min_total = torch.min(totals)
    print(totals)
    
    totals = torch.zeros(len(emotions), device=device).float()
    for x in indexes:
        row = torch.tensor([sheet.cell_value(x, 2 + j) for j in range(5)], device=device).float()
        
        if torch.max(totals + row) < min_total*(1 + allowed_exceedance):
            song.append(dataset.joinpath("{}.mp3".format(x)))
            emotion.append(row)
            totals += F.softmax(row)
            
    print(totals)
    return song, emotion
    
song, emotion = get_data(row_indexes)
# test_song, test_emotion = get_data(test_indexes)

train_song = [song[i] for i in range(len(song)) if i < len(song)*train_percentage]
test_song = [song[i] for i in range(len(song)) if i >= len(song)*train_percentage]

train_emotion = [emotion[i] for i in range(len(emotion)) if i < len(emotion)*train_percentage]
test_emotion = [emotion[i] for i in range(len(emotion)) if i >= len(emotion)*train_percentage]

num_maxes = [0 for _ in emotions]

for row in emotion:
    i = torch.argmax(row)
    num_maxes[i] += 1
    
print(num_maxes)

tensor([ 11.4343, 157.9053,  55.5535, 120.1634,  54.9434], device='cuda:0')
tensor([3.8678, 9.1242, 8.9825, 8.8231, 8.2024], device='cuda:0')
[1, 10, 10, 11, 7]


In [6]:
# train_emotion

In [7]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, mp3, labels, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.labels = labels
        self.mp3 = mp3
        self.cache = {}
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if index not in self.cache.keys():
#             print("index of " + str(index) + " was cached!")
            data, rate = librosa.load(self.mp3[index], sr=16000, duration=10)
            mfccs = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=40)
            assert rate == 16000
            sample_tensor = torch.tensor(mfccs, device=device).float()
            downsampled_tensor = sample_tensor[::10]
    #         print(mfccs.shape, data.shape)

            self.cache[index] = (downsampled_tensor, F.softmax(self.labels[index]))
#         else:
#             print("index was cached! index of " + str(index))
        return self.cache[index]

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (2,2))
        self.conv2 = nn.Conv2d(32, 64, (2,2))
        self.pool1 = nn.MaxPool2d((2,2))
        self.drop1 = nn.Dropout(p=0.25)
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(9920, 128)
        self.drop2 = nn.Dropout(p=0.5)
        self.dense2 = nn.Linear(128, 5)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = F.relu(x)
        
        x = self.drop2(x)
        x = self.dense2(x)
        x = F.relu(x)
        
        return F.log_softmax(x, dim = 1)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.25, inplace=False)
  (flat1): Flatten()
  (dense1): Linear(in_features=9920, out_features=128, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=128, out_features=5, bias=True)
)


In [9]:
train_set = SongEmotionDataset(train_song, train_emotion)
test_set = SongEmotionDataset(test_song, test_emotion)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

Train set size: 67
Test set size: 16


In [33]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data.unsqueeze_(1)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
#         output = output.view(-1, len(emotions))
#         print(output.shape, target.shape)
#         print(output, target)
#         loss = F.kl_div(output, target)
        kl = nn.KLDivLoss()
        loss = kl(output, target)
#         loss = F.cross_entropy(output, target)
#         loss = nn.CrossEntropyLoss(output, target)
#         loss = F.nll_loss(output, target)
        loss.backward()
        print(loss)
        optimizer.step()
#         scheduler.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [34]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data. unsqueeze_(1)
        output = model(data)
        print(output)
        print(target)
        print("\n")
#         output = output.permute(1, 0, 2)
        pred = output.max(1)[1] # get the index of the max log-probability 
        correct += pred.eq(target.max(1)[1]).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [35]:
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 100):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
tensor(0.2435, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 1 [0/67 (0%)]	Loss: 0.243493
tensor(0.1910, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2449, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1865, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2535, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2018, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 1 [40/67 (56%)]	Loss: 0.201776
tensor(0.2140, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1921, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1567, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],

tensor([[0.0157, 0.0426, 0.0426, 0.0426, 0.8564],
        [0.0408, 0.0150, 0.8185, 0.0150, 0.1108],
        [0.1643, 0.0604, 0.4466, 0.1643, 0.1643],
        [0.0117, 0.0861, 0.6364, 0.2341, 0.0317],
        [0.0388, 0.0388, 0.7784, 0.1053, 0.0388],
        [0.3483, 0.3483, 0.0471, 0.1281, 0.1281],
        [0.1441, 0.3917, 0.0530, 0.0195, 0.3917],
        [0.0317, 0.0861, 0.0117, 0.2341, 0.6364]], device='cuda:0')


tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.0146, 0.0397, 0.7979, 0.1080, 0.0397],
        [0.0202, 0.1491, 0.4053, 0.

tensor(0.2612, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.1441, 0.3917, 0.0530, 0.0195, 0.3917],
        [0.0047, 0.0345, 0.6931, 0.0127, 0.2550],
        [0.0388, 0.0388, 0.7784, 0.1053, 0.0388],
        [0.1643, 0.0604, 0.4466, 0.1643, 0.1643],
        [0.0146, 0.0397, 0.7979, 0.1080, 0.0397],
        [0.0408, 0.0150, 0.8185, 0.0150, 0.1108],
        [0.1915, 0.0705, 0.5206, 0.0259, 0.1915],
        [0.1044, 0.0141, 0.0052, 0.1044, 0.7718]], device='cuda:0')


tensor([[-1.6094, -1.6094, -

tensor(0.2871, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.3483, 0.3483, 0.0471, 0.1281, 0.1281],
        [0.1044, 0.0141, 0.0052, 0.1044, 0.7718],
        [0.0388, 0.0388, 0.7784, 0.1053, 0.0388],
        [0.1643, 0.0604, 0.4466, 0.1643, 0.1643],
        [0.6869, 0.0930, 0.0930, 0.0342, 0.0930],
        [0.0408, 0.0150, 0.8185, 0.0150, 0.1108],
        [0.0161, 0.0438, 0.0438, 0.0161, 0.8801],
        [0.0202, 0.1491, 0.4053, 0.0202, 0.4053]], device='cuda:0')


tensor([[-1.6094, -1.6094, -

tensor(0.2292, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 18 [40/67 (56%)]	Loss: 0.229240
tensor(0.2662, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1612, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1955, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.0317, 0.0861, 0.0117, 0.2341, 0.6364],
        [0.1643, 0.0604, 0.4466, 0.1643, 0.1643],
        [0.0408, 0.0150, 0.8185, 0.0150, 0.1108],
        [0.0202, 0.1491, 0.4053, 0.0202, 0.4053],
        [0.0161, 0.0438, 0.8801,

...

Train Epoch: 79 [0/67 (0%)]	Loss: 0.231582
tensor(0.1719, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1983, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2525, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1880, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2838, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 79 [40/67 (56%)]	Loss: 0.283792
tensor(0.1477, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2155, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2578, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0'

tensor(0.2411, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 83 [40/67 (56%)]	Loss: 0.241111
tensor(0.1865, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2259, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2714, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.1643, 0.0604, 0.4466, 0.1643, 0.1643],
        [0.0161, 0.0438, 0.8801, 0.0438, 0.0161],
        [0.1044, 0.0141, 0.0052, 0.1044, 0.7718],
        [0.0388, 0.0388, 0.7784, 0.1053, 0.0388],
        [0.1915, 0.0705, 0.5206,

tensor(0.1736, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1897, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.1044, 0.0141, 0.0052, 0.1044, 0.7718],
        [0.0117, 0.0861, 0.6364, 0.2341, 0.0317],
        [0.1643, 0.0604, 0.4466, 0.1643, 0.1643],
        [0.0317, 0.0861, 0.0117, 0.2341, 0.6364],
        [0.0161, 0.0438, 0.0438, 0.0161, 0.8801],
        [0.0161, 0.0438, 0.8801, 0.0438, 0.0161],
        [0.1441, 0.3917, 0.0530, 0.0195, 0.3917],
        [0.0047, 0.0345, 0.6931, 0.0127, 

tensor(0.1835, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2337, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2529, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 92 [40/67 (56%)]	Loss: 0.252929
tensor(0.1992, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2229, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2132, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward>)
tensor([[0.0117, 0.0861, 0.6364, 0.2341, 0.0317],
        [0.0317, 0.0861, 0.0117, 0.2341, 0.6364],
        [0.3483, 0

tensor(0.1864, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 96 [0/67 (0%)]	Loss: 0.186449
tensor(0.2172, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.1822, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2443, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2171, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2226, device='cuda:0', grad_fn=<KlDivBackward>)
Train Epoch: 96 [40/67 (56%)]	Loss: 0.222606
tensor(0.2170, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2150, device='cuda:0', grad_fn=<KlDivBackward>)
tensor(0.2246, device='cuda:0', grad_fn=<KlDivBackward>)
tensor([[-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.6094, -1.6094, -1.6094, -1.6094, -1.6094],
        [-1.60

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
torch.save(model.state_dict(), 'dataset_model_soundemotion.pt')

## NOTES

below is the mfccs notes / random code

In [13]:
audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# [print(x) for x in audio]

#convert audio into 2d array
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# mfccsscaled = np.mean(mfccs.T,axis=0)
print(mfccs.shape, audio.shape)
mfccs

/home/czf/memories/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(40, 2586) (1323648,)


array([[-5.30341797e+02, -4.07741577e+02, -3.27536621e+02, ...,
        -2.39811523e+02, -1.96744080e+02, -1.44711777e+02],
       [ 5.81265569e-01,  1.03006027e+02,  1.29354553e+02, ...,
         1.48707626e+02,  1.45873001e+02,  1.28202530e+02],
       [ 4.58764762e-01,  7.53921986e+00, -1.18814125e+01, ...,
        -2.51551704e+01, -1.92207527e+01, -1.79366188e+01],
       ...,
       [ 3.11299562e-01, -1.29907084e+00,  1.18818974e+00, ...,
        -6.58579540e+00, -3.34302998e+00, -4.75482178e+00],
       [ 2.23848164e-01, -3.19489312e+00, -2.78556681e+00, ...,
        -1.36089420e+01, -6.40699673e+00, -5.27228928e+00],
       [ 8.67742151e-02,  1.31472754e+00, -1.41885233e+00, ...,
         3.34440261e-01,  1.14392626e+00, -3.62402201e-02]], dtype=float32)

In [10]:
# audio_tensor = torch.tensor(audio)
# audio_tensor
# audio_tensor.shape

In [16]:
# for sound_file in data_path.iterdir():
#     if ".mp3" in str(sound_file):
#         print(sound_file)
#         audio, sample_rate = librosa.load(str(sound_file), res_type='kaiser_fast')
        
    

In [ ]:
# train_percentage = 0.8

# min_train = min_total*train_percentage
# min_test = min_total - min_train

# train_totals = torch.zeros(len(emotions))

# while 


# for i in range(1, 401):
#     count_total = sheet.cell_value(i, 7)
    
#     emotions_counter = [0 for e in emotions]
#     if i % 5 == 0:
#         test_song.append(dataset.joinpath("{}.mp3".format(i)))
#         emotion_arr = []
#         for j in range(5):
#             emotion_arr.append(sheet.cell_value(i, 2 + j))
#         test_emotion.append(torch.tensor(emotion_arr, device=device).float())
        
#     emotions_counter = [0 for e in emotions]
#     else:
#         train_song.append(dataset.joinpath("{}.mp3".format(i)))
#         emotion_arr = []
#         for j in range(5):
#             emotion_arr.append(sheet.cell_value(i, 2 + j))
#         train_emotion.append(torch.tensor(emotion_arr, device=device))

# print(len(train_song), len(test_song))
# print(len(train_emotion), len(test_emotion))